In [1]:
import pandas as pd
import pickle
data = pd.read_csv(r'quora1.csv')
print(data.shape)
print(data.info())

(404348, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404348 entries, 0 to 404347
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   total         404348 non-null  object
 1   is_duplicate  404348 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.2+ MB
None


## Dataset is taken from kaggle('Quora Question Pairs') then the columns 'question1' and 'question2'  are combined to form a column 'total'.

In [2]:
data.head()

,total,is_duplicate
0,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,0
2,How can I increase the speed of my internet co...,0
3,Why am I mentally very lonely? How can I solve...,0
4,"Which one dissolve in water quikly sugar, salt...",0


In [3]:
data = data[:303261]

In [4]:
print(data.shape)
print(data.info())

(303261, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303261 entries, 0 to 303260
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   total         303261 non-null  object
 1   is_duplicate  303261 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.6+ MB
None


In [5]:
data['is_duplicate'].value_counts()

0    190511
1    112750
Name: is_duplicate, dtype: int64

In [6]:
X = data['total']

In [7]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
lm = WordNetLemmatizer()
stopword  = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such']

In [8]:
corpus = []
for i in range(len(data)):
    review = re.sub('[^a-zA-Z]',' ',data['total'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(word) for word in review if not word in stopword]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=5000)
X = tf.fit_transform(corpus).toarray()

In [11]:
pickle.dump(tf,open('tfidf.pkl','wb'))

In [12]:
y = data['is_duplicate']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [14]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
ypred = classifier.predict(X_test)

In [15]:
import matplotlib.pyplot  as plt
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
cr = classification_report(y_test,ypred)
print(cr)
print("-------------------------------------------------------------------------------------")
cm = confusion_matrix(y_test,ypred)
print(cm)
print("-------------------------------------------------------------------------------------")
print(accuracy_score(y_test,ypred))

              precision    recall  f1-score   support

           0       0.73      0.90      0.81     47816
           1       0.71      0.43      0.54     28000

    accuracy                           0.73     75816
   macro avg       0.72      0.67      0.67     75816
weighted avg       0.72      0.73      0.71     75816

-------------------------------------------------------------------------------------
[[42958  4858]
 [15872 12128]]
-------------------------------------------------------------------------------------
0.7265748654637544


In [17]:
p_score = 0
import numpy as np
for alpha in np.arange(0,1,0.1):
    sub_classifier = MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    pred = sub_classifier.predict(X_test)
    score =accuracy_score(y_test,pred)
    if score>p_score:
        Classifier =sub_classifier
        print("ALPHA : {}, SCORE :{}".format(alpha,score))

C:\Users\LENOVO\anaconda3\envs\NLP\lib\site-packages\sklearn\naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


ALPHA : 0.0, SCORE :0.7271947873799726
ALPHA : 0.1, SCORE :0.7271156484119448
ALPHA : 0.2, SCORE :0.7270628890999261
ALPHA : 0.30000000000000004, SCORE :0.7270101297879076
ALPHA : 0.4, SCORE :0.7268386620238472
ALPHA : 0.5, SCORE :0.7268122823678379
ALPHA : 0.6000000000000001, SCORE :0.7267331433998101
ALPHA : 0.7000000000000001, SCORE :0.7266671942597869
ALPHA : 0.8, SCORE :0.7266540044317822
ALPHA : 0.9, SCORE :0.7266276247757729


In [18]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.9)
classifier.fit(X_train,y_train)
ypred = classifier.predict(X_test)
cr = classification_report(y_test,ypred)
print(cr)
print("-------------------------------------------------------------------------------------")
cm = confusion_matrix(y_test,ypred)
print(cm)
print("-------------------------------------------------------------------------------------")
print(accuracy_score(y_test,ypred))

              precision    recall  f1-score   support

           0       0.73      0.90      0.81     47816
           1       0.71      0.43      0.54     28000

    accuracy                           0.73     75816
   macro avg       0.72      0.67      0.67     75816
weighted avg       0.72      0.73      0.71     75816

-------------------------------------------------------------------------------------
[[42950  4866]
 [15860 12140]]
-------------------------------------------------------------------------------------
0.7266276247757729
